In [1]:
import pennylane as qml
import pennylane.numpy as np

from time import time

import matplotlib.pyplot as plt

from pennylane_lightning.lightning_tensor import LightningTensor

from datetime import datetime


/home/pietropaolo.frisoni/anaconda3/envs/PennyLane/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:34: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
def create_fun(n, depth, mode="state"):
    if mode == "mps":
        dev = LightningTensor(wires=qml.wires.Wires(range(n)))
    if mode == "default":
        dev = qml.device("default.qubit", wires=qml.wires.Wires(range(n)))

    def circuit(params):
        for i in range(n):
            qml.X(i)
        for ell in range(depth):
            for i in range(n-1):
                qml.CNOT(wires=[i, i+1])
            for i in range(n):
                qml.RX(params[ell, i], i)
        for i in range(n-1):
            qml.CNOT(wires=[i, i+1])
        return qml.expval(qml.Z(0))

    qnode = qml.QNode(circuit, dev)

    return qnode

In [3]:
def _timeit(callable, *args, reps=3):
    # callable.clear_cache()

    jittime0 = time()
    res0 = callable(*args)
    dt_jit = time() - jittime0 

    dts = []
    for k in range(reps):
        t0 = time()
        resi = callable(*args)
        dt = time() - t0

        dts.append(dt)
    
    return dt_jit, np.mean(dts), np.std(dts)

In [4]:
modes = ["mps", "default"]

In [5]:
n_wiress = np.arange(2, 20) # sizes to iterate over

datan = {}

for mode in modes:
    dt_jit = []
    dtss = []
    ddtss = []
    for i, n_wires in enumerate(n_wiress):
        depth = 20

        callable_ = create_fun(n_wires, depth, mode=mode)

        theta = np.random.rand(depth, n_wires)

        a, b, c = _timeit(callable_, theta)
        dt_jit.append(a)
        dtss.append(b)
        ddtss.append(c)

        print(f"mode: {mode} - {i+1} / {len(n_wiress)}, n_wires = {n_wires}")

    datan[mode] = dict(dtss=dtss, ddtss=ddtss, dt_jit=dt_jit, n_wiress=n_wiress)

mode: mps - 1 / 18, n_wires = 2
mode: mps - 2 / 18, n_wires = 3
mode: mps - 3 / 18, n_wires = 4
mode: mps - 4 / 18, n_wires = 5
mode: mps - 5 / 18, n_wires = 6
mode: mps - 6 / 18, n_wires = 7
mode: mps - 7 / 18, n_wires = 8
mode: mps - 8 / 18, n_wires = 9
mode: mps - 9 / 18, n_wires = 10
mode: mps - 10 / 18, n_wires = 11
mode: mps - 11 / 18, n_wires = 12
mode: mps - 12 / 18, n_wires = 13
mode: mps - 13 / 18, n_wires = 14
mode: mps - 14 / 18, n_wires = 15
mode: mps - 15 / 18, n_wires = 16
mode: mps - 16 / 18, n_wires = 17
mode: mps - 17 / 18, n_wires = 18
mode: mps - 18 / 18, n_wires = 19
mode: default - 1 / 18, n_wires = 2
mode: default - 2 / 18, n_wires = 3
mode: default - 3 / 18, n_wires = 4
mode: default - 4 / 18, n_wires = 5
mode: default - 5 / 18, n_wires = 6
mode: default - 6 / 18, n_wires = 7
mode: default - 7 / 18, n_wires = 8
mode: default - 8 / 18, n_wires = 9
mode: default - 9 / 18, n_wires = 10
mode: default - 10 / 18, n_wires = 11
mode: default - 11 / 18, n_wires = 12
mode

In [6]:
# datan

In [7]:
def expfit(x, y):
    y = np.log(y)
    coeff = np.polyfit(x, y, 1) # y=c1 e^(c0 x) => logy = c0 x + log(c1)
    return coeff

In [8]:
name = f"data/{datetime.now()}"
print(name)

data/2024-05-06 20:50:38.682740


In [9]:
np.savez(name, **datan)

In [10]:
# here you can re-load an older benchmark result
# name = data/2024-05-06 17:40:01.596466
data = np.load(name + ".npz", allow_pickle=True)

In [11]:
plot_modes = data.files

In [12]:
fig, axs = plt.subplots(ncols=2, figsize=(7,4), sharey=True)
color = ["tab:blue", "tab:orange", "tab:green", "black", "blue"] * 5
ax = axs[0]
for i, mode in enumerate(plot_modes):
    dat = data[mode].item()
    x, y = dat["n_wiress"][:], dat["dtss"][:]
    coeff = expfit(x, y)
    fit_string = f" {np.exp(coeff[1]):.2e} e^({coeff[0]:.2e} N)"
    ax.errorbar(dat["n_wiress"], dat["dtss"], dat["ddtss"], fmt="x:", label=mode, color=color[i])
    ax.plot(x, np.exp(coeff[1])*np.exp(coeff[0]*x), color=color[i], label=fit_string, alpha=0.5)
ax.set_yscale("log")
ax.set_ylabel("wallclock time")
ax.set_xlabel("N")
ax.legend(bbox_to_anchor=(0.5, 1), loc="lower center")

ax = axs[1]
for i, mode in enumerate(plot_modes):
    dat = data[mode].item()
    x, y = dat["n_wiress"][:], dat["dt_jit"][:]
    coeff = expfit(x, y)
    fit_string = f" {np.exp(coeff[1]):.2e} e^({coeff[0]:.2e} N)"
    ax.plot(dat["n_wiress"], dat["dt_jit"], "x:", color=color[i])
    ax.plot(x, np.exp(coeff[1])*np.exp(coeff[0]*x), color=color[i], label=fit_string, alpha=0.5)
ax.legend(bbox_to_anchor=(0.5, 1), loc="lower center")
ax.set_yscale("log")
ax.set_ylabel("first execution time")
ax.set_xlabel("N")

plt.savefig("plot_name.png")  # Change "plot_name.png" to the desired file name and format
plt.close()  # Close the plot to free up resources

Are the results the same?

In [13]:
n_wires=n_wiress[-1]

theta = np.random.rand(depth, n_wires)

mps_res = create_fun(n_wires, depth, mode="mps")(theta)
default_res = create_fun(n_wires, depth, mode="default")(theta)


In [14]:
mps_res, default_res

(-0.014337792400989202, tensor(-0.01433779, requires_grad=True))

In [15]:
n_wiress

tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19], requires_grad=True)